In [105]:
import pandas as pd
import numpy as np
import os
import openpyxl
import datetime
import parser

In [178]:
# Files Directions, Rutes and file names
filepathCostos = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\01. Costos'
filepathOtAv = r'\\u-srvnas\Areas\Mantenimiento\00.1 Mantto Planeamiento\0000. Gestión 2019\1. REPORTES\1. BASES DE DATOS DIARIOS\Data SAP\04. OT y Av'

#File Names
CeCo = r'CeCo1004.xlsx' #Estatico
ClaseCostos = r'KA23.xlsx' #Estatico
Budget = r'Presupuesto1004-2019Qlik.xlsx' #Estatico

filename3611 = r'3611.XLSX' #Dinamico
filename2993 = r'2993.XLSX' #Dinamico

#OT
filenameOt = r'IW39.xlsx'

In [179]:
#Charging DataSet
fullpath_CeCo = os.path.join(filepathCostos,CeCo)
fullpath_ClCo = os.path.join(filepathCostos,ClaseCostos)
fullpath_Budget = os.path.join(filepathCostos,Budget)

fullpath_3611 = os.path.join(filepathCostos,filename3611)
fullpath_2993 = os.path.join(filepathCostos,filename2993)
fullpath_Ot = os.path.join(filepathOtAv,filenameOt)

In [208]:
dbceco = pd.read_excel(fullpath_CeCo, converters={'CeCo':str})
dbclco = pd.read_excel(fullpath_ClCo, converters={'ClCo':str})
dbbudget = pd.read_excel(fullpath_Budget, converters={'CeCo':str, 'ClCo':str, 'MesNumero':str})

In [209]:
db3611 = pd.read_excel(fullpath_3611, converters={'Orden partner':str,
                                                  'Centro de coste':str,
                                                  'Clase de coste':str,
                                                  'Documento compras':str,
                                                  'Proveedor':str,
                                                  'Material':str})
db3611 = db3611.dropna(subset=['Centro de coste'])
db3611 = db3611.rename(columns={'Período':'Periodo',
                                'Fe.contabilización':'FechaContable',
                                'Valor/Moneda objeto':'Costo',
                                'Moneda del informe':'Moneda',
                                'Orden partner':'Orden',
                                'Denominación del objeto del interlocutor':'DenomOrden',
                                'Documento compras':'DocumentoCompras',
                                'Texto de pedido':'DenomDocCompras',
                                'Nombre':'DenomProveedor',
                                'Centro de coste':'CeCo',
                                'Denominación del objeto':'DenomCeCo',
                                'Clase de coste':'ClCo',
                                'Descrip.clases coste':'DenomClCo',
                                'Texto breve de material':'DenomMaterial',
                                'Cantidad total reg.':'CantMaterial',
                                'Clase de documento':'ClaseDocumento',
                                'Indic.cargo/abono':'IndCargoAbono'})
db3611['Escenario'] = 'Real'
db3611 = db3611.dropna(subset=['CeCo'])

In [210]:
db2993 = pd.read_excel(fullpath_2993, converters={'Orden':str,
                                                  'Clase de coste':str})
db2993 = db2993.dropna(subset=['Orden'])

db2993 = db2993.rename(columns={'División':'Division',
                                'Período':'Periodo',
                                'Fe.contabilización':'FechaContable',
                                'Denominación del objeto':'DenomOrden',
                                'Valor/Moneda objeto':'Costo',
                                'Moneda del objeto':'Moneda',
                                'Texto breve de material':'DenomMaterial',
                                'Cantidad total reg.':'CantMaterial',
                                'Ud. cantidad contab.':'UnidadCantMaterial',
                                'Documento compras':'DocumentoCompras',
                                'Texto de pedido':'DenomDocCompras',
                                'Clase de coste':'ClCo',
                                'Descrip.clases coste':'DenomClCo',
                                'Nombre':'DenomProveedor',
                                'Clase de documento':'ClaseDocumento',
                                'Indic.cargo/abono':'IndCargoAbono'})

db2993['Escenario'] = 'Real'

In [211]:
dbot = pd.read_excel(fullpath_Ot, head = 0,converters={'Orden':str,'Aviso':str,'Centro coste':str})
dbot = dbot.rename(columns={'Texto breve':'DenomOrden',
                                'Clase de orden':'ClaseOrden',
                                'Cl.actividad PM':'ClaseActividadOrden',
                                'Status sistema':'EstatusSistema',
                                'Status usuario':'EstatusUsuario',
                                'Indicador ABC':'CriticidadEq',
                                'Fe.inic.extrema':'FechaProgramada',
                                'Hora inicio':'HoraProgramada',
                                'Fe.inicio real':'FechaInicioEjecucion',
                                'Hora in.real':'HoraInicioEjecucion',
                                'Fecha fin real':'FechaFinEjecucion',
                                'Fin real (hora)':'HoraFinEjecucion',
                                'Liberación real':'FechaDeProgramacon',
                                'Denominación':'DenomEq',
                                'Ubicac.técnica':'UbicacionTecnica',
                                'Denominación.1':'DenmUbT',
                                'Plan mant.prev.':'PlanMtto',
                                'Revisión':'Revision',
                                'Centro coste':'CeCo',
                                'Grupo planif.':'GrupoPlanif',
                                'Pto.tbjo.resp.':'PuestoTrabajoResp',
                                'Puesto trabajo':'PuestoTrabajo'})


In [212]:
#Reporte de Costos / 2993 <-> Clase de Coste
clco2993 = pd.merge(db2993, 
                    dbclco[['ClCo', 'NaturalezaGasto']], 
                    on='ClCo', 
                    how = 'left',
                    validate = 'm:1')

In [213]:
#Reporte de Costos / 2993,clase de coste<-> Orden de Mantenimiento
clco2993ot = pd.merge(clco2993, 
                    dbot[['Orden',
                          'ClaseOrden',
                          'CeCo',
                          'CriticidadEq',
                          'Equipo','DenomEq',
                          'UbicacionTecnica', 
                          'DenmUbT',
                          'PlanMtto',
                          'GrupoPlanif']], 
                    on = 'Orden', 
                    how = 'left',
                    validate = 'm:1')

In [214]:
#Reporte de Costos / 2993,clase de coste,Orden de Mantenimiento<-> Centro de Coste
clco2993otceco = pd.merge(clco2993ot, 
                    dbceco[['CeCo','AreaResponsable','Proceso']],
                    on='CeCo',
                    how = 'left',
                    validate = 'm:1')
clco2993otceco["MesNumero"] = clco2993otceco["FechaContable"].apply(lambda x: x.strftime('%m'))

In [216]:
dbbudgetceco = pd.merge(dbbudget,
                    dbceco[['CeCo','AreaResponsable']],
                    on='CeCo',
                    how = 'left',
                    validate = 'm:1')

In [217]:
#Reporte de Costos / 3611 <-> Clase de Coste
clco3611 = pd.merge(db3611, 
                    dbclco[['ClCo', 
                            'NaturalezaGasto']], 
                    on='ClCo',
                    how = 'left',
                    validate = 'm:1')

In [218]:
#Reporte de Costos / 2993,clase de coste<-> Orden de Mantenimiento
clco3611ot = pd.merge(clco3611, 
                    dbot[['Orden',
                          'ClaseOrden',
                          'CriticidadEq',
                          'Equipo','DenomEq',
                          'UbicacionTecnica', 
                          'DenmUbT',
                          'PlanMtto',
                          'GrupoPlanif']], 
                    on='Orden', 
                    how = 'left',
                    validate = 'm:1')

In [219]:
#Reporte de Costos / 3611,clase de coste,Orden de Mantenimiento<-> Centro de Coste
clco3611otceco = pd.merge(clco3611ot, 
                    dbceco[['CeCo','AreaResponsable','Proceso']],
                    on='CeCo',
                    how = 'left',
                    validate = 'm:1')
clco3611otceco["MesNumero"] = clco3611otceco["FechaContable"].apply(lambda x: x.strftime('%m'))

In [220]:
dbbudgetceco.columns

Index(['Escenario', 'CeCo', 'DenomCeCo', 'Division', 'Proceso', 'Año',
       'MesNumero', 'Mes', 'NaturalezaGasto', 'ClCo', 'DenomClCo', 'Costo',
       'AreaResponsable'],
      dtype='object')

In [221]:
RC02 = pd.concat([clco3611otceco[['FechaContable', 
                                  'Orden',
                                  'DenomOrden', 
                                  'Costo',
                                  'ClaseDocumento', 
                                  'IndCargoAbono', 
                                  'Escenario', 
                                  'NaturalezaGasto',
                                  'CeCo',
                                  'AreaResponsable', 
                                  'Proceso', 
                                  'MesNumero']], 
                  dbbudgetceco[['CeCo', 
                            'MesNumero',
                            'Costo',
                            'Proceso',
                            'AreaResponsable',
                            'Escenario',
                            'NaturalezaGasto']]], 
                 ignore_index=True, 
                 sort=True)

In [173]:
with pd.ExcelWriter('output.xlsx', mode='w') as writer:
    dbbudgetceco.to_excel(writer, sheet_name='Sheetname2',index = False)